In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/2021W51 Input.csv')
df.head()

,OrderID,Customer,Order Date,Return State,Category,Sub-Category,Product Name,Unit Price,Quantity
0,Edinburgh-1347,Mr Schneider,24/01/2021,NaN,Furniture,Bookcases,Bookcase,£60.07,7
1,Newcastle-1924,Mrs Balk,15/01/2021,NaN,Furniture,Bookcases,Bookcase,£60.07,7
2,London-2097,Mrs Foster,07/01/2021,NaN,Furniture,Bookcases,Bookcase,£60.07,10
3,London-1704,Mr Badders,14/01/2021,NaN,Furniture,Chairs,Armless Task Chair,£37.75,11
4,Bristol-1911,Mr Norvell,23/01/2021,Return Processed,Furniture,Chairs,Armless Task Chair,£37.75,4


In [12]:
# Split out the store name from the OrderID
df[['Store', 'OrderID']] = df['OrderID'].str.split('-', expand=True)


# When assigning IDs, these should be created using the dimension and minimum order date fields so that the IDs do not change when later orders are placed
# For the Customer dimension table, we want to include additional fields detailing their total number of orders and the % of products they have returned
# Replace the dimensions with their IDs in the original dataset to create the fact table

In [17]:
# Turn the Return State field into a binary Returned field
df['Returned'] = np.where(df['Return State'].isna(), 0, 1)

In [35]:
# Create a Sales field
df['sales'] = df['Unit Price'].str.strip('£').astype('float') * df['Quantity']

In [51]:
# Create 3 dimension tables for Store, Customer and Product
# Create Store table
df_store = df.groupby('Store')['Order Date'].min().reset_index()\
            .sort_values(by=['Order Date','Store'])\
            .rename(columns={'Order Date': 'First Order'})
df_store['StoreID'] = range(1, len(df_store)+1)

In [57]:
# create Customer table
df_customer = df.groupby('Customer').agg(Returned=('Returned', 'sum'),
                                        Order_Lines=('OrderID', 'count'),
                                        Number_of_Order=('OrderID', 'nunique'),
                                        First_order=('Order Date', 'min')).reset_index()\
                .sort_values(by=['First_order', 'Customer'])

In [63]:
df_customer['Return %'] = df_customer['Returned'] / df_customer['Order_Lines']
df_customer['Return %'] = df_customer['Return %'].round(2)

In [64]:
df_customer['CustomerID'] = range(1, len(df_customer)+1)

In [72]:
# create Product table
df_product = df.groupby(['Product Name', 'Sub-Category', 'Category']).agg(Unit_price=('Unit Price', 'min'),
                                                                          First_Sold=('Order Date', 'min')).reset_index()\
                .sort_values(by=['First_Sold', 'Product Name'])
df_product['productID'] = range(1, len(df_product)+1)

In [75]:
display(df_store.head())
display(df_customer.head())
display(df_product.head())

,Store,First Order,StoreID
0,Birmingham,01/01/2021,1
4,Manchester,01/01/2021,2
5,Newcastle,01/01/2021,3
1,Bristol,01/04/2021,4
2,Edinburgh,01/04/2021,5


,Customer,Returned,Order_Lines,Number_of_Order,First_order,Return %,CustomerID
11,Mr Barnes,4,11,4,01/01/2021,0.36,1
238,Mr Wiediger,0,2,1,01/01/2021,0.00,2
346,Mrs Philippe,3,9,4,01/01/2021,0.33,3
8,Mr Baird,4,10,4,01/04/2021,0.40,4
9,Mr Ballard,0,4,2,01/04/2021,0.00,5


,Product Name,Sub-Category,Category,Unit_price,First_Sold,productID
27,Post-its,Art,Office Supplies,£3.33,01/01/2021,1
39,Stacking Storage Drawers,Storage,Office Supplies,£43.26,01/01/2021,2
53,Xerox 1979,Paper,Office Supplies,£6.45,01/01/2021,3
56,Xerox 208,Paper,Office Supplies,£7.14,01/01/2021,4
0,Armless Task Chair,Chairs,Furniture,£37.75,01/04/2021,5
